In [23]:
import os
import plotly
import pandas as pd
import numpy as np
import plotly.plotly as py
import plotly.figure_factory as ff
import plotly.graph_objs as go
import collections
import re
from operator import itemgetter

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

plotly.tools.set_credentials_file(username='Fernet', api_key='zC7ogVfVT5UPcpu2VKse')
plotly.offline.init_notebook_mode(connected=True)

#---------------- Settings ----------------

image_ext = "svg" #Extension para bajar los graficos
poda = 5 #Cuantos datos se dejan a cada lado al podar
image_h = 1000
image_w = 1400

#---------------- Funciones ----------------


#Toma el archivo a leer y si se quiere ordenar por tam o por tiempo promedio. Por default, se ordenan por num_tira

#Experimentos 1.1, 2.1 y 3.1, n variable
def procesar_csv_nVar(data_path):
    df = pd.read_csv(data_path, sep = ";")
    cant_exp = df.shape[0] #Shape = (filas, columnas)
    proms = []
    #Ordeno las instancias por tiempo
    df.sort_values("Tiempo", inplace = "True")
    #Descarto los valores en los extremos
    df = df.iloc[poda : cant_exp - poda, :]

    promedio = df["Tiempo"].mean() #Promedio podado de las instancias
    varianza = df["Tiempo"].var()  #Varianza de los tiempos
    v = df["V"].iloc[0]    
    
    
    ret_data = v, promedio, varianza
    
    return ret_data

def procesar_carpeta_nVar(folder_path):
    tam = []
    proms = []
    vars = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".csv"):
                file = folder_path + "/" + file
                f = open(file, 'r')
                datos = procesar_csv_nVar(file)
                tam.append(datos[0])
                proms.append(datos[1])
                vars.append(datos[2])
                f.close()
    
    dict = {"Tam": tam, "Proms": proms, "Vars": vars}
    df = pd.DataFrame(data = dict)
    df.sort_values("Tam", inplace = True)
    return df

#Experimentos 1.2, 3.2, %k utilizables o %const variable
def procesar_csv_pctVar(data_path, var):  #var es string. Ejs: pctConst, pctK
    df = pd.read_csv(data_path, sep = ";")
    cant_exp = df.shape[0] #Shape = (filas, columnas)
    pct_prom = df[var].mean().round()
    proms = []
    #Ordeno las instancias por tiempo
    df.sort_values("Tiempo", inplace = "True")
    #Descarto los valores en los extremos
    df = df.iloc[poda : cant_exp - poda, :]

    promedio = df["Tiempo"].mean() #Promedio podado de las instancias
    
    ret_data = pct_prom, promedio
    
    return ret_data

def procesar_carpeta_pctVar(folder_path, var):
    pctVar = []
    proms = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".csv"):
                file = folder_path + "/" + file
                f = open(file, 'r')
                datos = procesar_csv_pctVar(file, var)
                pctVar.append(datos[0])
                proms.append(datos[1])
                f.close()
    
    dict = {var: pctVar, "Proms": proms}
    df = pd.DataFrame(data = dict)
    df.sort_values(var, inplace = True)
    return df      

#Exp 1.1
def procesaro_archivo_ej1_exp1(data_path):
    df = pd.read_csv(data_path, sep = ";")
    cant_exp = df.shape[0] #Shape = (filas, columnas)
    proms = []
    #Ordeno las instancias por tiempo
    df.sort_values("Tiempo", inplace = "True")
    #Descarto los valores en los extremos
    df = df.iloc[poda : cant_exp - poda, :]

    promedio = df["Tiempo"].mean() #Promedio podado de las instancias
    
    return promedio

def procesar_carpeta_ej1_exp1(folder_path):
    #Settings
    buenosCasos = []
    malosCasos = []
    cant_caminos = [5, 10, 15, 20, 25, 30, 35, 40, 50, 75, 80, 95, 100]
    largo_camino = 20
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".csv"):
                file = folder_path + "/" + file
                f = open(file, 'r')
                prom = procesaro_archivo_ej1_exp1(file)
                caminos_exp = [int(s) for s in re.split('[_ .]', file)  if s.isdigit()]
                caminos_exp
                if "buenCaso" in file:
                    buenosCasos.append((prom, caminos_exp))
                else:
                    malosCasos.append((prom, caminos_exp))
                f.close()
    
    buenosCasos = sorted(buenosCasos, key = itemgetter(1))
    malosCasos = sorted(malosCasos, key = itemgetter(1))
    
    return cant_caminos, largo_camino, buenosCasos, malosCasos

#Toma un vector de tuplas (x, y, nombre) y devuelve una figura con cada tupla de datos
def crear_fig(traces, xtitle, ytitle, fig_title):
    data = []
    for trace in traces:
        #Para que todas las traces sean distintas ademas del color
        i = 0
        modo = str(i)
        i += 1

        new_trace = go.Scatter(
            x = trace[0],
            y = trace[1],
            name = trace[2],
            mode = modo
                
        )
        data.append(new_trace)
    
    layout = go.Layout(
        xaxis=dict(
            tickmode = "auto",
            #nticks = cant_exp,
            ticks = "inside",
            #tick0 = 0,
            #ticklen = 8,
            #dtick = 3,
            title = xtitle,
            tickangle = 0
        ),
        yaxis=dict(
            tick0 = 0,
            zeroline = "False",
            ticks='inside',
            ticklen=8,
            tickwidth=3,
            tickcolor='#000',
            title = ytitle
        ),
        legend=dict(
            x=10,
            y=1,
            traceorder='normal',
            font=dict(
                family='sans-serif',
                size=12,
                color='#000'
            ),
            bgcolor='#E2E2E2',
            bordercolor='#FFFFFF',
            borderwidth=2
        ),
        title = fig_title
    )
    
    return go.Figure(data=data, layout=layout)

#---------------- Script ----------------

#Procesamiento de datos

data_ej1_exp1 = procesar_carpeta_ej1_exp1("Outputs/Ej1/outsTiempo/exp1")
ej1_exp1_cantCaminos =  data_ej1_exp1[0]
ej1_exp1_largoCaminos = data_ej1_exp1[1]
ej1_exp1_buenosCasos =  [tiempo[0] for tiempo in data_ej1_exp1[2]]
ej1_exp1_malosCasos =  [tiempo[0] for tiempo in data_ej1_exp1[3]]


data_ejer1_exp2 = procesar_carpeta_pctVar("Outputs/Ej1/outsTiempo/exp2", "pctK")
ejer1_exp2_pctK =  data_ejer1_exp2["pctK"]
ejer1_exp2_proms =  data_ejer1_exp2["Proms"]

data_ejer2_exp1 = procesar_carpeta_nVar("Outputs/Ej2/outsTiempo/exp1")
ejer2_exp1_tams =  data_ejer2_exp1["Tam"]
ejer2_exp1_proms =  data_ejer2_exp1["Proms"]

data_ejer3_exp1 = procesar_carpeta_nVar("Outputs/Ej3/outsTiempo/exp1")
ejer3_exp1_tams =  data_ejer3_exp1["Tam"]
ejer3_exp1_proms =  data_ejer3_exp1["Proms"]

data_ejer3_exp2 = procesar_carpeta_pctVar("Outputs/Ej3/outsTiempo/exp2", "pctConst")
ejer3_exp2_pctConst =  data_ejer3_exp2["pctConst"]
ejer3_exp2_proms =  data_ejer3_exp2["Proms"]

# #Traces
ejer1_exp1_buenosCasos_trace = (ej1_exp1_cantCaminos, ej1_exp1_buenosCasos, "ejer1_exp1_buenosCasos")
ejer1_exp1_malosCasos_trace = (ej1_exp1_cantCaminos, ej1_exp1_malosCasos, "ejer1_exp1_malosCasos")


ejer1_exp2_trace = (ejer1_exp2_pctK, ejer2_exp1_proms, "ejer1_exp2")

ejer2_exp1_trace = (ejer2_exp1_tams, ejer2_exp1_proms, "ejer2_exp1")

ejer3_exp1_trace = (ejer3_exp1_tams, ejer3_exp1_proms, "ejer3_exp1")
ejer3_exp2_trace = (ejer3_exp2_pctConst, ejer3_exp2_proms, "ejer3_exp2")


#Graficos

#Exp 1.1
nombre = "ejer1_exp1 <br> Largo de los caminos = %d" %ej1_exp1_largoCaminos

traces = [ejer1_exp1_buenosCasos_trace, ejer1_exp1_malosCasos_trace]

fig = crear_fig(traces, "Cantidad de caminos", "Tiempo promedio (ms)", nombre)

plotly.offline.iplot(fig)

#Exp 1.2
nombre = "ejer1_exp2"

traces = [ejer1_exp2_trace]

fig = crear_fig(traces, "Porcentaje de premiums utilizables", "Tiempo promedio (ms)", nombre)

plotly.offline.iplot(fig)

#Exp 2.1
nombre = "ejer2_exp1"

traces = [ejer2_exp1_trace]

fig = crear_fig(traces, "Cantidad de vertices", "Tiempo promedio (ms)", nombre)

plotly.offline.iplot(fig)

#Exp 3.1
nombre = "ejer3_exp1"

traces = [ejer3_exp1_trace]

fig = crear_fig(traces, "Cantidad de vertices", "Tiempo promedio (ms)", nombre)

plotly.offline.iplot(fig)

#Exp 3.2
nombre = "ejer3_exp2 <br> n = 100"

traces = [ejer3_exp2_trace]

fig = crear_fig(traces, "Procentaje de rutas construidas", "Tiempo promedio (ms)", nombre)

plotly.offline.iplot(fig)



[311086.70000000001,
 466438.04999999999,
 753056.32499999995,
 1109279.2250000001,
 1505904.95,
 1935885.8500000001,
 2439966.375,
 3016560.8500000001,
 4150210.3999999999,
 7956493.9000000004,
 8859797.1750000007,
 11843651.550000001,
 12902483.625]

[486855.5,
 1845368.925,
 4092114.7999999998,
 7210127.875,
 11227283.199999999,
 16314347.525,
 22167973.475000001,
 28607047.125,
 44602138.524999999,
 99883505.575000003,
 113547732.15000001,
 161009354.65000001,
 177205809.90000001]

In [15]:
import re
file = "hola_exp1_ejer2_5.csv"
split = )
split

['hola', 'exp1', 'ejer2', '5', 'csv']